In [ ]:
from main_library import *
%reload_ext dotenv
%dotenv
%reload_ext autoreload
%autoreload 2

In [ ]:
extractDate = date_load()
proj = projects_load()
proj_id_signed = proj.project_id.unique()

prop = proposals_load()
proj = proj_add_cols(prop, proj)

stage_p =  ['REJECTED' ,'NO_MONEY' ,'MAIN', 'RESERVE', 'INELIGIBLE', 'WITHDRAWN', 'INADMISSIBLE', None]
prop1 = proposals_status(prop, proj_id_signed, stage_p) 
call_to_integrate, call_miss = proposals_id_missing(prop1, proj, extractDate)

# project data missing in proposals if call already in proposals then add this
proj1 = proj_id_miss_fixed(prop1, proj, call_to_integrate)
call_miss = list(set(call_miss)-set(call_to_integrate))
proj = proj.loc[~proj.callId.isin(call_miss)]

# merge proj + prop
print('### MERGED PROPOSALS/PROJECTS')
if len(proj1)==0:
    prop2=pd.concat([proj,prop1], ignore_index= True)
else:
    prop2 = pd.concat([prop1, proj1, proj], ignore_index = True)

prop2 = prop2.loc[~((prop2.status_code=='REJECTED')&(prop2.stage=='successful'))]
print(f"- result - merged all: {len(prop2)},\n{prop2[['stage','status_code']].value_counts()}")
merged = copy.deepcopy(prop2)
merged = dates_year(merged)
merged = strings_v(merged)
merged = url_to_clean(merged)
merged.mask(merged=='', inplace=True)
merged = empty_str_to_none(merged)      
merged.rename(columns={
    'freekw':'free_keywords',
    'callDeadlineDate':'call_deadline', 
    'callId':'call_id', 
    'submissionDate':'submission_date',
    'startDate':'start_date',
    'endDate':'end_date', 
    'ecSignatureDate':'signature_date'}, inplace=True)

if any(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1]):
    print(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1])

merged = merged_panels(merged)
merged = merged_topics(merged)
merged = merged_actions(merged)

# calls list
calls = call(PATH_SOURCE+FRAMEWORK+'/')
print("\n### CALLS+MERGED")
if len(merged.loc[merged.call_id.isnull()])>0:
        print(f"1 - ATTENTION : manque des call_id: {merged.loc[merged.call_id.isnull(), 'project_id']}")
else:
    call_id = merged[['call_id', 'call_deadline']].drop_duplicates()
    print(f"2 - CALL_ID de merged -> nb call+deadline: {len(call_id)}, nb call unique: {call_id.call_id.nunique()} ")

calls = calls_to_check(calls, call_id)

projects = projects_complete_cleaned(merged, extractDate)

In [ ]:
##### PARTICIPATIONS
part = participants_load(proj)
# conserve uniquement les projets présents dans proposals et applicants
part = part.loc[part.project_id.isin(projects.project_id.unique())]
print(f"- size part hors proj manquant: {len(part)}")
part = part_role_type(part)
part = erc_role(part, projects)

#### APPLICANTS
app = applicants_load(prop)
# conserve uniquement les projets présents dans proposals et applicants
app1 = app.loc[app.project_id.isin(projects.project_id.unique())] 
print(f"- size app1 hors proj exclus: {len(app1)}")

app_missing_pid = projects.loc[(projects.stage=='evaluated')&(~projects.project_id.isin(app1.project_id.unique())), 'project_id'].unique()
tmp = part[part.project_id.isin(app_missing_pid)]
app1 = part_miss_app(tmp, app1)

#redressement accelerator
acc_folio = pd.read_csv(f"{PATH_SOURCE}{FRAMEWORK}/eic_fund_portfolio.csv", sep=';', dtype={'PROPOSAL_NBR':str})
print(f"size acc_folio: {len(acc_folio)}")
acc = (app1.loc[(app1.project_id.isin(acc_folio.PROPOSAL_NBR.unique()))&(app1.role=='Coordinator'),['project_id', 'role']]
       .merge(acc_folio[['PROPOSAL_NBR','GRANT_REQUESTED']], how='inner', left_on='project_id', right_on='PROPOSAL_NBR')
       .drop(columns='PROPOSAL_NBR'))
print(f"size acc: {len(acc)}")
app1 = app1.merge(acc, how='left', on=['project_id', 'role'])
app1.loc[app1.requestedGrant.isnull(), 'requestedGrant'] = app1.GRANT_REQUESTED
app1.drop(columns=['GRANT_REQUESTED'], inplace=True)

app1 = app_role_type(app1)
app1 = erc_role(app1, projects)

del app

####
# verification Etat des participations
part = check_multiP_by_proj(part)
app1 = check_multiA_by_proj(app1)


In [ ]:
def merged_partApp(app1, part):
    import pandas as pd, numpy as np
    print("\n### create LIEN")

    part2 = part[['project_id', 'orderNumber', 'generalPic', 'participant_pic', 'n_part']].drop_duplicates()
    app2 = app1[['project_id', 'orderNumber', 'generalPic', 'participant_pic', 'n_app']].drop_duplicates()
    print(f'app2: {len(app2)} part2:{len(part2)}')
    cols_part = part2.columns
    cols_app = app2.columns

    '''proposal uniquement'''
    lien1 = (app2
            .merge(part2[['project_id']], how='outer', on='project_id', indicator=True)
            .query('_merge == "left_only"')
            .drop('_merge', axis=1)
            .assign(base_only='prop_only'))
    app2 = (app2.merge(lien1[['project_id']], how='outer', on='project_id', indicator=True)
            .query('_merge == "left_only"')
            .drop('_merge', axis=1))
    print(f'- applicant uniquement lien1: {len(lien1)} reste à croiser -> app2: {len(app2)}')

    '''jointure parfaite'''
    lien2 = part2.merge(app2, how='inner')
    print(f'jointure parfaite -> lien2: {len(lien2)}')

    part3 = (part2.merge(lien2[cols_part], how='outer', indicator=True)
             .query('_merge == "left_only"')
             .drop('_merge', axis=1))
    app3 = (app2.merge(lien2[cols_app], how='outer', indicator=True)
            .query('_merge == "left_only"')
            .drop('_merge', axis=1))
    print(f'reste à croiser -> app3: {len(app3)} part3: {len(part3)}')

    '''jointure sans orderNumber'''
    lien3 = part3.merge(app3, how='inner', on=['project_id', 'generalPic', 'participant_pic'], suffixes=('', '_p'))
    print(f'jointure sans ordernumber -> lien3: {len(lien3)}')

    part4 = part3.merge(lien3[cols_part], how='outer', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
    df_app = lien3[['project_id', 'generalPic', 'participant_pic', 'orderNumber_p']].rename(columns=({'orderNumber_p':'orderNumber'}))
    app4 = (app3.merge(df_app, how='outer', indicator=True)
            .query('_merge == "left_only"')
            .drop('_merge', axis=1))
    print(f'reste à croiser -> app4: {len(app4)} part4: {len(part4)}')


    '''jointure sans participant_pic'''
    lien4 = part4.merge(app4, how='inner', on=['project_id', 'generalPic', 'orderNumber'], suffixes=('', '_p'))
    print(f'jointure sans participant_pic -> lien4: {len(lien4)}')

    part5 = (part4.merge(lien4[cols_part], how='outer', indicator=True)
             .query('_merge == "left_only"')
             .drop('_merge', axis=1))
    df_app = lien4[['project_id', 'generalPic', 'participant_pic_p', 'orderNumber']].rename(columns=({'participant_pic_p':'participant_pic'}))
    app5 = (app4.merge(df_app, how='outer', indicator=True)
            .query('_merge == "left_only"')
            .drop('_merge', axis=1))
    print(f'reste à croiser -> app5: {len(app5)} part5: {len(part5)}')

    '''jointure juste generalPic'''
    lien5 = part5.merge(app5, how='inner', on=['project_id', 'generalPic'], suffixes=('', '_p'))
    print(f'jointure seulement avec generalpic -> lien5: {len(lien5)}')

    if len(lien5)>0:
        part6 = (part5.merge(lien5[cols_part], how='outer', indicator=True)
                 .query('_merge == "left_only"')
                 .drop('_merge', axis=1))
        df_app = lien5[['project_id', 'generalPic', 'participant_pic_p', 'orderNumber_p']].rename(columns=({'orderNumber_p':'orderNumber', 'participant_pic_p':'participant_pic'}))
        app6 = (app5.merge(df_app, how='outer', indicator=True)
                .query('_merge == "left_only"')
                .drop('_merge', axis=1))
        print(f'reste à croiser -> app6: {len(app6)} part6: {len(part6)}')
        
        '''jointure juste participant_pic'''
        lien6 = (part6
                .merge(app6, how='inner', on=['project_id', 'participant_pic'], suffixes=('', '_p'))
                .assign(base_only='a_joindre'))
        print(f'jointure avec seulement participant_pic -> lien6: {len(lien6)}')
    
        if len(lien6)>0:
            print('code pour ajouter lien6 à la table lien finale')
    else:
        '''jointure juste participant_pic'''
        lien6 = part5.merge(app5, how='inner', on=['project_id', 'participant_pic'], suffixes=('', '_p'))
        print(f'jointure avec seulement participant_pic -> lien6: {len(lien6)}')
        if len(lien6)>0:
            print('code pour ajouter lien6 à la table lien finale')


    lien = pd.concat([lien1, lien2, lien3, lien4, app5, part5], ignore_index=True).drop_duplicates()
    lien = lien.assign(inProposal=np.where(~lien['n_part'].isnull() & lien['n_app'].isnull(), False, True))
    lien = lien.assign(inProject=np.where(lien['n_part'].isnull() & ~lien['n_app'].isnull(), False, True))

    lien['orderNumber_p'] = np.where((lien['orderNumber_p'].isnull()) & (lien['inProposal']==True), lien['orderNumber'], lien['orderNumber_p'])
    lien['orderNumber'] = np.where(lien['inProject']==False, None, lien['orderNumber'])

    lien['participant_pic_p'] = np.where((lien['participant_pic_p'].isnull()) & (lien['inProposal']==True), lien['participant_pic'], lien['participant_pic_p'])
    lien['participant_pic'] = np.where(lien['inProject']==False, None, lien['participant_pic'])

    for x in lien.columns:
        if pd.api.types.infer_dtype(lien[x])=='string':
            lien.loc[:,x]=np.where(lien.loc[:,x].isnull(), None, lien.loc[:,x])
        
    lien.columns = ['proposal_'+k[0:-2] if k[-2:] == '_p' else k for k in list(lien.columns)]
    lien['calculated_pic'] = np.where(~lien['participant_pic'].isnull(), lien['participant_pic'], lien['proposal_participant_pic'])

    lien['projNlien'] = lien.groupby(['project_id', 'proposal_orderNumber', 'generalPic', 'calculated_pic'], dropna = False).pipe(lambda x: x.orderNumber.transform('nunique'))
    lien['propNlien'] = lien.groupby(['project_id', 'orderNumber', 'generalPic', 'calculated_pic'], dropna = False).pipe(lambda x: x.proposal_orderNumber.transform('nunique'))
    lien.loc[lien['projNlien']==0, 'projNlien']=1
    lien.loc[lien['propNlien']==0, 'propNlien']=1

    print(f'- size lien: {len(lien)}') 
    length_lien=len(lien)
    # print(lien[lien['n_lien']>1])

    lien.loc[lien.inProject==True, 'participation_linked'] = lien['project_id']+"-"+lien['orderNumber']
    lien.loc[lien.inProposal==True, 'participation_linked'] = lien['project_id']+"-"+lien['proposal_orderNumber']
    
    # add countryCode
    lien = (lien
            .merge(part[['project_id', 'orderNumber', 'generalPic', 'participant_pic', 'countryCode']],
                   how='left', on=['project_id', 'orderNumber', 'generalPic', 'participant_pic']))

    lien = (lien
            .merge(app1[['project_id', 'orderNumber', 'generalPic', 'participant_pic', 'countryCode']], 
                   how='left', left_on=['project_id', 'proposal_orderNumber', 'generalPic', 'proposal_participant_pic'],
                   right_on=['project_id', 'orderNumber', 'generalPic', 'participant_pic'],
                   suffixes=[ '','.y'])
            .drop(columns=[ 'participant_pic.y', 'orderNumber.y'])
            .rename(columns={'countryCode.y':'proposal_countryCode'}))

    lien.loc[lien.countryCode.isnull(), 'countryCode'] = lien.loc[lien.countryCode.isnull(), 'proposal_countryCode']

    if any(lien.countryCode.isnull()):
        print(f"- ATTENTION {lien[lien.countryCode.isnull()].generalPic.nunique()} countryCode missing {lien[lien.countryCode.isnull()].generalPic.unique()}")

    # verif que chaque obs contient un calculated pic
    lien_no_pic=len(lien[lien['calculated_pic'].isnull()])
    if lien_no_pic > 0:
        print(f'1- attention {lien_no_pic} entités dans la table LIEN n\'ont pas de pic secondaire' )

    del app2, app3, app4, app5, app6, part2, part3, part4, part5, part6, lien1, lien2, lien3, lien4, lien5, lien6
    return lien



lien = merged_partApp(app1, part)

In [ ]:
lien.head(5
          )